# Decision Tree

# Import Statements

In [67]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

# Load and Prepare Data

### Format of the data¶
    - last coloumn of the data frame must contain the label and it must also be called label
    - there should be no missing values in the data frame

In [68]:
df = pd.read_csv("Iris.csv")
df = df.drop("Id",axis = 1)
df = df.rename(columns= {"species":"label"})

In [166]:
#df.head()

In [167]:
#df.info()

In [71]:
def train_test_split(df,test_size):
    
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df,test_df

In [168]:
#random.seed(0)
#train_df,test_df = train_test_split(df,test_size= 20)

# Helper Functions

In [169]:
data = train_df.values
#data[:5]

### Data pure?

In [74]:
def check_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

### Classify

In [75]:
def classify_data(data):
    
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column,return_counts= True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

### Potential slits?

In [170]:
#train_df.head()

In [85]:
def get_potential_splits(data):
    
    potential_splits = {}
    _,n_columns = data.shape

    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:,column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index-1]
                potential_split =(current_value + previous_value ) / 2

                potential_splits[column_index].append(potential_split)

    return potential_splits

In [87]:
#potential_splits = get_potential_splits(train_df.values)         

In [94]:
#sns.lmplot(data=train_df,x = "petal_width", y = "petal_length",hue = "label",fit_reg = False, size = 6, aspect = 1.5)

#plt.vlines(x = potential_splits[3], ymin=1,ymax= 7)
#plt.hlines(y = potential_splits[2], xmin=0,xmax= 2.5)

### Split Data

In [144]:
def split_data(data,split_column,split_value):
    
    split_column_values = data[:,split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
                      
    return data_below,data_above

In [145]:
split_column = 3
split_value = 0.8

In [146]:
data_below,data_above = split_data(data,split_column,split_value)

In [147]:
#plotting_df = pd.DataFrame(data_below,columns= df.columns)
#sns.lmplot(data=plotting_df,x = "petal_width",y = "petal_length",fit_reg = False, size = 6, aspect = 1.5)
#plt.vlines(x = split_value, ymin= 1, ymax= 7)
#plt.xlim(0,2.6)

### Lowest Overall Entropy

In [148]:
def calculate_entropy(data):
    
    label_column = data[:,-1]
    _,counts = np.unique(label_column,return_counts = True)

    probabilities = counts/counts.sum()
    entropy = sum(probabilities* -np.log2(probabilities))
    
    return entropy

In [149]:
def calculate_overall_entropy(data_below,data_above):
    
    n_data_points = len(data_below) + len(data_above)

    p_data_below= len(data_below)/n_data_points
    p_data_above= len(data_above)/n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below) + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [171]:
#calculate_overall_entropy(data_below,data_above)

In [158]:
def determine_best_split(data,potential_splits):
    
    overall_entropy = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below,data_above = split_data(data,split_column = column_index,split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below,data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value

    return best_split_column,best_split_value

In [172]:
#potential_splits = get_potential_splits(data)

In [173]:
#determine_best_split(data,potential_splits)

# Decision Tree Algorithm

dictionary = {question : [yes_answer,no_answer]}

example_tree = {"petal_width <= 0.8":["Iris-setosa",
                                     {"petal_width <= 1.65": [{"petal_width <= 4.9" : ["Iris-versicolor",
                                                                                      "Iris-virginica"]},
                                                             "Iris-virginica"]}]}

### Algorithm

In [190]:
def decision_tree_algorithm(df,counter = 0,min_samples = 2 , ):
    
    # data preparations
    if counter == 0:
        global column_headers
        column_headers = df.columns
        data = df.values
    else:
        data = df
        
    # base case
    if (check_purity(data)) or (len(data) < min_samples):
        classification = classify_data(data)
        return classification
    
    # recursive part
    else:
        counter += 1
        
        # helper functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data,potential_splits)
        data_below,data_above = split_data(data,split_column,split_value)
        
        # instantiate subtree
        feature_name = column_headers[split_column]
        question = "{} <= {}".format(feature_name,split_value)
        subtree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below,counter, min_samples)
        no_answer = decision_tree_algorithm(data_above,counter, min_samples)
        
        subtree[question].append(yes_answer)
        subtree[question].append(no_answer)
        
        return subtree
        

In [191]:
tree = decision_tree_algorithm(train_df, min_samples=60)
pprint(tree)

{'petal_width <= 0.8': ['Iris-setosa',
                        {'petal_width <= 1.65': ['Iris-versicolor',
                                                 'Iris-virginica']}]}


In [180]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label'], dtype='object')